In [ ]:
import time, DM_Profiles, numpy, scipy
import pynbody as pyn
from pynbody import units
from matplotlib import pylab as plt
s = pyn.load('/media/tengiz/81498267-fb7c-4587-88ea-5c63aaf2cb66/Pynbody/testdata/g15784.lr.01024.gz')
s.physical_units()
h = s.halos()

In [ ]:
def model_prep(halo):
    """
    This prepares profile for model to take in
    """
    # centering to generate profile and placing particles back 
    with pyn.analysis.angmom.faceon(halo, cen_size  =  '1 kpc'):
        r_200 = float(pyn.analysis.halo.virial_radius(halo.d, overden = 200))
        stellar_profile = pyn.analysis.profile.Profile(halo.s, min = 0.01, max = r_200, ndim = 2, type = 'equaln', nbins = 10000)
        shm_radius = stellar_profile['rbins'][len(stellar_profile['rbins'])//2]
#         profile = pyn.analysis.profile.Profile(halo.d, min = 2*max(halo.d['eps']), max = 0.2*r_200, ndim = 3, type = 'log', nbins = 50)
        profile = pyn.analysis.profile.Profile(halo.d, min = 10, max = 0.2*r_200, ndim = 3, type = 'log', nbins = 50)
    
    # calculating steallar and halo mass
    
    sm = halo.s['mass'].sum()
    hm = halo['mass'].sum()
    t_sf = halo.properties['time'].in_units('Gyr')
        
    return profile, sm, hm, shm_radius, r_200, t_sf

In [ ]:
def den_plot(halo_profile, to_save = False):
    '''
    takes model profile object from DM_Profiles and plots den profile with a curve_fit
    '''
    
    fig, ax = plt.subplots() 
    ax.plot(halo_profile.radii, numpy.power(10, halo_profile.log_den), 'g--')
    ax.plot(halo_profile.radii, numpy.power(10, halo_profile.log_rho(numpy.array(halo_profile.radii), *halo_profile.params)), 'r-')
    ax.grid()
    ax.legend(('data','fit'))
    ax.set_title(halo_profile.name + ' ' + halo_profile.pmodel + ' density profile')
    ax.set_xlabel('$R$ [kpc]')
    ax.set_ylabel(r'$\rho$ [M$_{\odot}$ /kpc$^{3}$]')
    plt.errorbar(halo_profile.radii, numpy.power(10, halo_profile.log_den) , yerr = numpy.array(halo_profile.den_error), fmt = 'none')
    ax.set_yscale('log')
    ax.set_xscale('log')
    if to_save:
        plt.savefig('../Graphs/density_plot' + halo_profile.name + '' + halo_profile.pmodel +  '_density.jpg')
    plt.show()

In [ ]:
i = 1
halo = h[i]
start_time = time.time()
variables = model_prep(halo) 

In [ ]:
for k in DM_Profiles.models():
    hp = DM_Profiles.model(*variables, 'halo_' +  str(i), pmodel = k)
    print(hp.output())
    den_plot(hp, to_save = False)
print(time.time() - start_time)

In [ ]:
C_200list = []
e_C_200list = []
for k in DM_Profiles.models()[1:]:
    hp = DM_Profiles.model(*variables, 'halo_' +  str(i), pmodel = k)
    C_200, r_s, e_r_s = hp.output()[0][1], hp.output()[1][1], numpy.sqrt(hp.output()[2][1][1])
    e_C_200 = hp.r_200/r_s**2*e_r_s
    C_200list.append(C_200)
    e_C_200list.append(e_C_200)
    print('for ' + k + ' profile \t' + 'C_200 = ' + str(C_200) + '\t +/- \t' + str(e_C_200))

In [ ]:
lables = DM_Profiles.models()[1:]
x = range(1,len(C_200list)+1)
plt.bar(x, C_200list, color = 'blue')
plt.errorbar(x, C_200list, yerr = e_C_200list, color = 'red', fmt = 'none')
plt.xticks(x, lables)
plt.yscale('linear')
plt.show()